In [1]:
# general packages
import numpy as np
import pandas as pd
import datatable as dt

## Data preposessing

In [2]:
%%time
#parse_dates = ['CreatedDate', 'RequestedDate', 'AccTime', 'PupTime', 'DelTime']
#A = pd.read_csv('Booking_Dispatch_training_set.csv', parse_dates = parse_dates, 
#date_parser = lambda x: pd.to_datetime(x).strftime("%Y-%m-%d %H:%M:%S"))
#data_parsed = A.to_csv('data_parsed.csv')

Wall time: 0 ns


In [3]:
%%time
#parse_dates = ['CreatedDate', 'RequestedDate']
#B = pd.read_csv('Booking_Dispatch_training_set.csv', parse_dates = parse_dates, 
#date_parser = lambda x: pd.to_datetime(x).strftime("%Y-%m-%d %H:%M:%S"))
#data_parsed = B.to_csv('val_data_parsed.csv')

Wall time: 0 ns


In [ ]:
%%time
train_data_datatable = dt.fread('data_parsed.csv') 
data = train_data_datatable.to_pandas()
data = data.iloc[:,1:]

In [ ]:
%%time
val_data_datatable = dt.fread('val_data_parsed.csv')
val_data = val_data_datatable.to_pandas()
val_data = val_data.iloc[:,1:]

In [ ]:
data.shape

In [ ]:
val_data.shape

In [ ]:
#subtract the features and target variable for modeling 
train_features = data[['BookingSource','Conditions',
     'HasCondition','IsAccountBooking','CreatedDate','RequestedDate',
     'BookingFleet','DispatchFleet','AreaNumber','PuPlace','PuAddress',
     'PuSuburb','PuLat','PuLong','TargetVariable']]

In [ ]:
val_features = val_data[['BookingSource','Conditions',
     'HasCondition','IsAccountBooking','CreatedDate','RequestedDate',
     'BookingFleet','DispatchFleet','AreaNumber','PuPlace','PuAddress',
     'PuSuburb','PuLat','PuLong','TargetVariable']]

In [ ]:
train_features.head()

In [ ]:
val_features.head()

In [ ]:
#concatenate training data and validation data for feature engineering
features = pd.concat([train_features, val_features], axis=0)

In [ ]:
features.head()

In [ ]:
# check datatypes
features.dtypes

In [ ]:
#BookingSource variable
Conditions_count = data['Conditions'].value_counts().rename_axis('unique_values').reset_index(name='counts')
Conditions_count

In [ ]:
feature1 = features

## Feature engineering

In [ ]:
# BookingSource 
dummies = pd.get_dummies(features1['BookingSource'], drop_first=False)
feature1 = feature1.join(dummies)
feature1 = feature1.drop('room_type', axis= 1)

## Modeling